In [1]:
#@Imports

#############################
# region: Imports and Setup #
#############################
import sys
import os
path = os.path.abspath('../..')
if path not in sys.path:
  sys.path.insert(0, path)

%load_ext autoreload
#############################
# endregion                 #
#############################

import datetime
from datetime import datetime as dt
from datetime import timedelta as td

from concordia.associative_memory import importance_function, associative_memory
from concordia.components import agent as components
from concordia.typing import agent as simulacrum_agent

from examples.custom_components.setup import model_setup, measurement_setup
from examples.tpb import utils
from examples.tpb import components as tpb_components
from examples.tpb import memory as tpb_memory
from examples.tpb import plan as tpb_plan
from examples.tpb import agent as tpb_agent

In [2]:
SETUP_TIME=dt(year=2024,month=10,day=1,hour=18)
TIME_STEP=td(minutes=5)

model, embedder = model_setup("llama3")
measurement, clock = measurement_setup(setup_time=SETUP_TIME,time_step=TIME_STEP)

#@title Importance models
importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()

/Users/rgelpi/.pyenv/versions/3.12.2/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/rgelpi/.pyenv/versions/3.12.2/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
config = tpb_agent.AgentConfig(
  name="Alice",
  gender="female",
  context="Alice is a university student who has just signed up for a psychology experiment about fitness.",
  goal="Alice wants to graduate from her degree",
  traits="Determined, scatterbrained, picky",
  extras={
    'model': model,
    'embedder': embedder,
    'importance': importance_model.importance,
    'clock_now': clock.now
    }
)

/Users/rgelpi/Documents/GitHub/concordia/concordia/associative_memory/associative_memory.py:107: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self._memory_bank = pd.concat(
/Users/rgelpi/.pyenv/versions/3.12.2/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


As a young girl, Alice was determined to learn how to ride a bike without training wheels. She would spend hours in her backyard, trying and failing, until finally, she found herself soaring down the sidewalk on two wheels. This early victory set the tone for her future endeavors - she would not settle for mediocrity, and she would push herself to achieve her goals, no matter how scatterbrained she might be.

As Alice grew older, her determination only intensified. She became a university student, juggling classes with part-time jobs and extracurricular activities. Her pickiness extended to every aspect of her life, from the type of coffee she drank to the books she read. But despite her many quirks, Alice was drawn to the psychology experiment about fitness, hoping to gain insight into the human mind and perhaps even unlock some secrets to improve her own physical performance.

Years went by, and Alice became a successful researcher in her field. Her days were filled with data analysi

In [4]:
params = {
  "model": model,
  "config": config,
  "num_memories": 100,
  "clock_now": clock.now,
  "verbose": True,
  "num_behavs": 5,
  "num_people": 5,
  "threading": True
}

# Memory
memory = tpb_memory.EpisodicMemory(**params)

# Situation
situation = components.situation_perception.SituationPerception(
  "situation",
  model=model,
  memory=config.memory,
  agent_name=config.name,
  clock_now = clock.now,
  num_memories_to_retrieve=100,
  verbose=True
)

# TPB pipeline components
behav = tpb_components.Behaviour("behaviour", **params)
attitude = tpb_components.Attitude("attitude", components=[behav], **params)
people = tpb_components.People("people", components=[behav], **params)
motiv = tpb_components.Motivation("motivation", components=[people], **params)
norm = tpb_components.SubjectiveNorm("norm", components=[motiv], **params)
control = tpb_components.BehaviouralControl("control", components=[behav], **params)
thin_goal = tpb_components.ThinGoal("thin_goal", **params)
intention = tpb_components.BehaviouralIntention(
  "intention",
  components=[attitude, norm, control, thin_goal],
  **params
)
plan = tpb_plan.TPBPlan(
  "plan",
  goal = intention,
  components=[situation],
  **params
)

full_model = tpb_components.TPBModel(
  "full_model",
  components=[
    memory,
    behav,
    attitude,
    people,
    motiv,
    norm,
    control,
    thin_goal,
    situation,
    intention,
    plan
  ]
)

In [5]:
alice = tpb_agent.TPBAgent(
  config=config,
  full_model=full_model,
  clock=clock
)

[LOG] [Component] behaviour initialized.
[LOG] [Component] attitude initialized.
[LOG] [Component] people initialized.
[LOG] [Component] motivation initialized.
[LOG] [Component] control initialized.
[LOG] [Component] intention initialized.
[LOG] [Component] thin_goal initialized.
As Alice deliberated on whether to take the job offer, she weighed the potential benefits of increased responsibility and financial stability against the risks of burnout and compromised work-life balance, considering how these consequences might impact not only her own life but also those around her, including her family and colleagues.a university student who has just signed up for a psychology experiment about fitness.Memories of Alice:
[23 Jul 1986 00:00:00] When Alice was six years old, she tried to ride her friend Emma's tricycle without permission. She loved the feeling of freedom and speed, but Emma got upset when Alice crashed into a bush. This early experience taught Alice that sometimes taking risk

In [6]:
from concordia.typing import scene as scene_lib

CALL_TO_ACTION = (
    "Given the above, what will {agent_name} do for the "
    "next {timedelta} to achieve their current plan?"
  )

scenes = [
  scene_lib.SceneSpec(
    scene_type=scene_lib.SceneTypeSpec(
      "gym_priming",
      premise={"Alice": [
        "Alice is working in the mail room today. ",
        "The mail room is across the street from the gym. "
        ]},
      action_spec=simulacrum_agent.ActionSpec(call_to_action=CALL_TO_ACTION,output_type="FREE")
    ),
    start_time=clock.now(),
    participant_configs=[config],
    num_rounds=1
  )
]

In [7]:
# @title Create the game master object
game_master_memory = associative_memory.AssociativeMemory(
   sentence_embedder=embedder,
   importance=importance_model_gm.importance,
   clock=clock.now)

for premise in [
    config.context
  ]:
  game_master_memory.add(premise)

from concordia.components import game_master as gm_components

relevant_events = gm_components.relevant_events.RelevantEvents(
  clock_now=clock.now,
  model=model,
  memory=game_master_memory,
)

from concordia.environment import game_master
from concordia.thought_chains import thought_chains
env = game_master.GameMaster(
    model=model,
    memory=game_master_memory,
    clock=clock,
    players=[alice],
    components=[relevant_events],
    update_thought_chain=(thought_chains.attempt_to_result, thought_chains.result_to_effect_caused_by_active_player,),
    randomise_initiative=False,
    player_observes_event=True,
    players_act_simultaneously=False,
    verbose=True,
)

/Users/rgelpi/Documents/GitHub/concordia/concordia/associative_memory/associative_memory.py:107: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self._memory_bank = pd.concat(


In [8]:
from concordia.environment.scenes import runner

runner.run_scenes(
  env,
  scenes,
  [alice],
  clock,
  verbose=True
)



    Scene 0    Participants: ['Alice']

Alice -- premise: Alice is working in the mail room today. 
Alice -- premise: The mail room is across the street from the gym. 
Based on Alice's traits and goals, here are five potential behaviors she can take in response to the situation:

1. **Take a break**: Given that Alice is scatterbrained, she might need some time to collect her thoughts before tackling the experiment. She could take a short walk or do something else to clear her mind.
2. **Review the experiment details**: As someone who wants to learn more about psychology and fitness, Alice might want to review the experiment's objectives and requirements to ensure she understands what's expected of her.
3. **Start tracking her activities**: Building on her previous experience with tracking daily activities and eating habits, Alice could start monitoring her own physical performance and habits to gain a better understanding of how she can improve.
4. **Seek guidance from her instructor

## Structured Example of TPB Agent

### Pre-action memories (Premise)
- Character's backstory (university students participating in a research study)
- Character sees a link sent out on her university listserv inviting students to participate in a research study.
- Character clicks the link and begins the study.

### Survey section
- Character reads the study instructions.
- Survey contains the following sections:
  - Normative beliefs intervention
  - Control beliefs interventions
  - Attitude intervention
  - For each section, ask:
    1. Were you aware of this? (yes/no)
    2. How important is this for you? (1-7 ranking)
- Note: survey is done under interrogation before each scene.

### Scene section
- Example scenes: health vs. not, cue to gym vs. not, ease of getting to gym vs. not, subjective norms vs. not, control beliefs vs. not
- Action goes into memory, as well as outcome of action.
- DV for each scene:
  - Did "go to the gym" appear as an action?
  - What was the behavioural intention (softmax) if it did appear?
  - What was ultimately chosen as the action?
  - Correlation of attitudes and subjective norms and behavioural control with interrogation scale
  - Fill scale out AFTER doing TPB.
- Action chosen and the result of the action are recorded after each scene and go into memory.

In [9]:
from examples.tpb import quiz

quiz = quiz.Quiz(
  model=model,
  config=config,
  questions=r"""
{
  "questions": [
    {
      "question": "What is the largest moon of the planet Jupiter?",
      "options": [
        "Ganymede",
        "Pluto",
        "Io",
        "Enceladus",
        "Europa"
      ]
    }
  ]
}
"""
)

In [10]:
observation = (
  "Alice is taking a quiz just for fun. She is trying to guess the "
  "right answer to the questions she sees on the quiz. If she isn't sure, she will guess. "
  "What choice would Alice make to answer the following question? "
  "Answer with only the letter of the option, e.g., a), b), c), or d). "
  "Do not provide any explanation or description, just the letter answer. "
)

quiz.observe(observation)

b)

In [11]:
quiz.results

{'What is the capital of the planet Jupiter?': 0}